In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

from summer.utils import ref_times_to_dti

from autumn.tools.plots.utils import REF_DATE
from autumn.tools.runs import ManagedRun
from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt
from autumn.settings.region import Region

## Specify the runs of interest

In [ ]:
run_id = "covid_19/myanmar/1635935706/37ea4d8"

In [ ]:
plot_left_date = datetime.date(2020, 9, 1)

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
full_run = mr.full_run.get_derived_outputs()
pbi = mr.powerbi.get_db()
targets = pbi.get_targets()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 20), sharex="all")
outputs = ("notifications", "infection_deaths")
for i_ax, axis in enumerate(axes.reshape(-1)):
    output = outputs[i_ax]
    scenario_chain = (full_run["scenario"] == 0) & (full_run["chain"] == 0)
    for i_run in full_run[scenario_chain]["run"].unique():
        selection = full_run[(full_run["run"] == i_run) & scenario_chain]
        axis.plot(ref_times_to_dti(REF_DATE, selection["times"]), selection[output])
    if output in targets:
        axis.scatter(targets.index, targets[output], color="k", s=10, alpha=0.5, zorder=10)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)

In [ ]:
def plot_outputs(output, scenario):
    fig, axes = plt.subplots(4, 2, figsize=(12, 20), sharex="all")
    colour = (0.2, 0.2, 0.8)
    for i_ax, axis in enumerate(axes.reshape(-1)):
        region = Region.VICTORIA_SUBREGIONS[i_ax]
        results_df = results[region][(output, scenario)]
        indices = results_df.index
        axis.fill_between(indices, results_df[0.01], results_df[0.99], color=colour, alpha=0.1)
        axis.fill_between(indices, results_df[0.025], results_df[0.975], color=colour, alpha=0.5)
        axis.fill_between(indices, results_df[0.25], results_df[0.75], color=colour, alpha=0.7)
        axis.plot(indices, results_df[0.500], color="b", linestyle="--")
        if output in targets[region]:
            axis.scatter(targets[region].index, targets[region][output], color="k", alpha=0.5, s=10)
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(region)
    fig.tight_layout()